In [1]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import utils

# Concat

In [2]:
def _concat(xs):
    """nd tensor to 1d tensor

    Args:
        xs (array): the array of nd tensor

    Returns:
        array: concated array
    """
    return tf.concat([tf.reshape(x, [tf.size(x)]) for x in xs], axis=0, name="_concat")

## Testing

In [3]:
a = tf.constant([[[[1],[2],[3]], [[4], [5], [6]]], [[[2],[4],[6]], [[8], [10], [12]]]])
b = tf.constant([1,2])

In [4]:
tf.concat(tf.reshape(a, [tf.size(a)]), axis=-1)

<tf.Tensor 'concat:0' shape=(12,) dtype=int32>

# Architect

In [28]:
class Architect(object):
    """Constructs the model

    Parameters:
      network_momentum(float):  network momentum
      network_weight_decay(float): network weight decay
      model(Network): Network archtecture with cells
      optimise(optimiser): Adam / SGD
    """

    def __init__(self, model, args):
        """Initialises the architecture

        Args:
            model (Network): Network archtecture with cells
            args (dict): cli args
        """
        self.network_momentum = args.momentum
        self.network_weight_decay = args.weight_decay
        self.model = model
        self.arch_learning_rate = args.arch_learning_rate
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.arch_learning_rate,
                                                beta1=0.5,
                                                beta2=0.999)    
        self.learning_rate = args.learning_rate

    def get_model_theta(self, model):
        specific_tensor = []
        specific_tensor_name = []
        for var in model.trainable_weights:
            if not 'alphas' in var.name:
                specific_tensor.append(var)
                specific_tensor_name.append(var.name)
        return specific_tensor
    
    def step(self, input_train, target_train, input_valid, target_valid, lr, unrolled):
        """Computer a step for gradient descend

        Args:
            input_train (tensor): a train of input
            target_train (tensor): a train of targets
            input_valid (tensor): a train of validation
            target_valid (tensor): a train of validation targets
            eta (tensor): eta
            network_optimizer (optimiser): network optimiser for network
            unrolled (bool): True if training we need unrolled
        """
        if unrolled:
#             w_regularization_loss = tf.add_n(utils.get_var(tf.losses.get_regularization_losses(),'network')[1])
            w_regularization_loss = 0.25
            logits = self.model(input_train)
            train_loss = self.model._loss(logits, target_train)
            train_loss += 1e4*0.25*w_regularization_loss
            lr = 0.001
            return self._compute_unrolled_step(input_train, 
                                               target_train, 
                                               input_valid, 
                                               target_valid,
                                               self.get_model_theta(self.model),
                                               train_loss,
                                               lr)
        else:
            return self._backward_step(input_valid, target_valid)
        
    
    def _compute_unrolled_step(self, x_train, y_train, x_valid, y_valid, w_var, train_loss, lr):
        arch_var = self.model.arch_parameters()
        
        unrolled_model = self.model.new()
        logits = unrolled_model(x_train)
        unrolled_train_loss = unrolled_model._loss(logits, y_train)  
        unrolled_w_var = self.get_model_theta(unrolled_model)
        copy_weight_opts = [v.assign(w) for v,w in zip(unrolled_w_var,w_var)]
        #w'
        with tf.control_dependencies(copy_weight_opts):
            unrolled_optimizer = tf.train.GradientDescentOptimizer(lr)
            unrolled_optimizer = unrolled_optimizer.minimize(unrolled_train_loss, var_list=unrolled_w_var)

        valid_logits = unrolled_model(x_valid)
        valid_loss = unrolled_model._loss(valid_logits, y_valid)
        tf.summary.scalar('valid_loss', valid_loss)

        with tf.control_dependencies([unrolled_optimizer]):
            valid_grads = tf.gradients(valid_loss, unrolled_w_var)

        r=1e-2
        R = r / (tf.global_norm(valid_grads)+1e-6)

        optimizer_pos=tf.train.GradientDescentOptimizer(R)
        optimizer_pos=optimizer_pos.apply_gradients(zip(valid_grads, w_var))

        optimizer_neg=tf.train.GradientDescentOptimizer(-2*R)
        optimizer_neg=optimizer_neg.apply_gradients(zip(valid_grads, w_var))

        optimizer_back=tf.train.GradientDescentOptimizer(R)
        optimizer_back=optimizer_back.apply_gradients(zip(valid_grads, w_var))

        with tf.control_dependencies([optimizer_pos]):
            train_grads_pos=tf.gradients(train_loss, arch_var)
            with tf.control_dependencies([optimizer_neg]):
                train_grads_neg=tf.gradients(train_loss,arch_var)	
                with tf.control_dependencies([optimizer_back]):
                  leader_opt= self.optimizer
                  leader_grads=leader_opt.compute_gradients(valid_loss, var_list =unrolled_model.arch_parameters())
        for i,(g,v) in enumerate(leader_grads):
            leader_grads[i]=(g - self.learning_rate * tf.divide(train_grads_pos[i]-train_grads_neg[i],2*R),v)

        leader_opt=leader_opt.apply_gradients(leader_grads)
        return leader_grads, unrolled_model
    
    def _backward_step(self, input_valid, target_valid):
        """Backward step for validation

        Args:
            input_train (tensor): a train of input
            target_train (tensor): a train of targets
        """
        loss = self.model._loss(self.model(input_valid), target_valid)
        opt = self.optimiser.minimize(loss, var_list=model.get_weights())
        return opt

## Testing

In [6]:
from model_search import Network

In [7]:
criterion = tf.losses.sigmoid_cross_entropy
model = Network(3, 3, criterion)
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "arch_weight_decay": 1e-3
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

W0915 19:01:40.655931 140564929759040 deprecation.py:506] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0915 19:01:40.661149 140564929759040 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:61: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0915 19:01:40.662756 140564929759040 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:67: The name tf.layers.BatchNormalization is deprecated. Please use tf.compat.v1.layers.BatchNormalization instead.

W0915 19:01:40.666864 140564929759040 deprecation_wrapper.py:119] From /home/mythrex/Coding/Darts-Unet/cnn/operations.py:7: The na

In [8]:
inp = tf.random_uniform((1, 2, 2, 3), 0, 255)
target = tf.random_uniform((1, 2, 2, 1), 0, 1)
input_valid = tf.random_uniform((1, 2, 2, 3), 0, 255)
target_valid = tf.random_uniform((1, 2, 2, 1), 0, 1)
lr=0.025
unrolled=True

In [9]:
image = tf.random_uniform((1, 2, 2, 3), 0, 255)
init = tf.global_variables_initializer()
res = model(image)

W0915 19:01:43.679900 140564929759040 deprecation.py:323] From /home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [29]:
architect = Architect(model, Struct(**args))

In [30]:
opt = architect.step(inp, target, input_valid, target_valid, lr, unrolled)

In [31]:
with tf.Session() as sess:
    sess.run(init)
    tf.initialize_all_variables().run()
    out = sess.run(opt)

AbortedError: Operation received an exception:Status: 5, message: could not create a view primitive descriptor, in file tensorflow/core/kernels/mkl_slice_op.cc:433
	 [[node gradients_1/network_7_1/cell_31/concat_grad/Slice_3 (defined at <ipython-input-28-2c2d37217741>:75) ]]

Original stack trace for 'gradients_1/network_7_1/cell_31/concat_grad/Slice_3':
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mythrex/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
  